##Importing the Libraried

In [0]:
!pip install keras_metrics

In [0]:
import tqdm
import numpy as np
import keras_metrics # for recall and precision metrics
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding, LSTM, Dropout, Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.model_selection import train_test_split
import time
import numpy as np
import pickle

## Downloading the Data

In [0]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv

--2020-02-15 04:55:27--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv [following]
--2020-02-15 04:55:27--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uccfbf2d2baf5ca5860e439cbc1a.dl.dropboxusercontent.com/cd/0/inline/AyL6lxABU5xmWGJ8-fC3bPftEU1RYPbQ7DDgA1PbFPV1u0UQfP0_zT-J8hDVqoTi6eEkwZ5_vU4GVDzhAdLDGhIjZ5Uitpo5gd0QeQi0iKYJqQ/file# [following]
--2020-02-15 04:55:28--  https://uccfbf2d2baf5ca5860e439cbc1a.dl.dropboxusercontent.com/cd/0/inline/AyL6lxABU5xmWGJ8-fC3bPftEU1RYPbQ7DDgA1PbFPV1u0UQfP0_zT-J8hDVqoT

## Glove file

In [0]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-02-15 04:55:33--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-02-15 04:55:33--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-02-15 04:55:33--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


## Function for getting word vector from Glove

In [0]:
def get_embedding_vectors(tokenizer, dim=300):
    embedding_index = {}
    with open(f"glove.6B.{dim}d.txt", encoding='utf8') as f:
        for line in tqdm.tqdm(f, "Reading GloVe"):
            values = line.split()
            word = values[0]
            vectors = np.asarray(values[1:], dtype='float32')
            embedding_index[word] = vectors

    word_index = tokenizer.word_index
    embedding_matrix = np.zeros((len(word_index)+1, dim))
    for word, i in word_index.items():
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            # words not found will be 0s
            embedding_matrix[i] = embedding_vector
            
    return embedding_matrix

## Loading the Data

In [0]:
import pandas as pd
data=pd.read_csv("train.csv")

In [0]:
data.shape

(1306122, 3)

In [0]:
data.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


## Target Distribution

In [0]:
data.target.value_counts()

0    1225312
1      80810
Name: target, dtype: int64

In [0]:
data.drop("qid",axis=1,inplace=True)

In [0]:
X=data.question_text
y=data.target

In [0]:
X[0]

'How did Quebec nationalists see their province as a nation in the 1960s?'

In [0]:
from nltk import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
sent_len=[len(word_tokenize(x)) for x in X] ## Length of all Question_text in file

In [0]:
max(sent_len) ## Max len of Question in file

412

In [0]:
np.percentile(sent_len,90)

25.0

In [0]:
np.percentile(sent_len,95)

30.0

## Tokenizing and Padding of Question

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Seq_len = 30
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
# convert to sequence of integers
X = tokenizer.texts_to_sequences(X)

X = np.array(X)
y = np.array(y)

X = pad_sequences(X, maxlen=Seq_len)



In [0]:
X[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    9,   48, 6683, 7219,  158,
         55, 6107,   36,    4, 1206,    6,    1, 8333], dtype=int32)

## Getting class weight for Imbalanced Distribution

In [0]:
from sklearn.utils import class_weight
class_weight=class_weight.compute_class_weight('balanced',np.unique(y),y)

In [0]:
class_weight

array([0.53297527, 8.08143794])

In [0]:
np.unique(y,return_counts=True)

(array([0, 1]), array([1225312,   80810]))

In [0]:
y[0:200]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [0]:
y = to_categorical(y)

In [0]:
y.shape

(1306122, 2)

In [0]:
print(y[0])

[1. 0.]


In [0]:

X.shape

(1306122, 30)

## Splitting the Data

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)


In [0]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

((1175509, 30), (1175509, 2), (130613, 30), (130613, 2))

## Getting the Glove vector

In [0]:
embedding_matrix = get_embedding_vectors(tokenizer)

Reading GloVe: 400000it [00:32, 12267.88it/s]


In [0]:
type(embedding_matrix)

In [0]:
from keras.layers import Flatten, LSTM


## Model Architecture

In [0]:
model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1,
              300,
              weights=[embedding_matrix],
              trainable=False,
              input_length=30))
model.add(LSTM(128, recurrent_dropout=0.2,return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(64, recurrent_dropout=0.2,return_sequences=True))
model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Dense(2, activation="softmax"))
model.compile(optimizer="adam", loss="categorical_crossentropy",
                  metrics=["accuracy", keras_metrics.precision(), keras_metrics.recall()])
model.summary()












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 300)           66648600  
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           219648    
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 64)            49408     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                614

## Training Model

In [0]:
model.fit(x_train, y_train, validation_data=(x_test, y_test),
          batch_size=512, epochs=10,
          class_weight={0:0.53297527,1:8.08143794},verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 1175509 samples, validate on 130613 samples
Epoch 1/10
1175509/1175509 [==============================] - 285s 243us/step - loss: 0.3021 - acc: 0.8700 - precision: 0.9915 - recall: 0.8688 - val_loss: 0.3364 - val_acc: 0.8536 - val_precision: 0.9944 - val_recall: 0.8486
Epoch 2/10
1175509/1175509 [==============================] - 284s 241us/step - loss: 0.2575 - acc: 0.8886 - precision: 0.9935 - recall: 0.8870 - val_loss: 0.2270 - val_acc: 0.9044 - val_precision: 0.9924 - val_recall: 0.9049
Epoch 3/10
1175509/1175509 [==============================] - 283s 240us/step - loss: 0.2383 - acc: 0.8958 - precision: 0.9943 - recall: 0.8941 - val_loss: 0.2073 - val_acc: 0.9153 - val_precision: 0.9916 - val_recall: 0.9174
Epoch 4/10
1175509/1175509 [==============================] - 282s 240us/step - loss: 0.2212 - acc: 0.9007 - precision: 0.9950 - recall: 0.8987 - val_loss: 0.2456 - val_acc: 

## Evaluation of Model

In [59]:
pred=model.predict(x_test)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,pred)

0.958721972517711

## Saving Model

In [0]:
from keras.models import load_model
model.save("Spam.h5")